In [1]:
import pandas as pd

from HestonModel import Heston, HestonPriceFunction

heston = Heston()
heston.load_data('Data/20230912_PLIQ_IP.csv')
heston.preprocess_data()
heston.get_from_market()
heston.optimize_params()
heston.yearly_historical_volatility

Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid parameters
Invalid para

KeyboardInterrupt: 

In [ ]:
import yfinance as yf
yf.download(ticker_symbol, )

In [78]:
from datetime import timedelta
# Definir el ticker para Google

ticker_symbol = 'AAPL'
ticker_data = yf.Ticker(ticker_symbol)
options_expirations = ticker_data.options
expiration_date = options_expirations[2]
options_chain = ticker_data.option_chain(expiration_date)
calls = options_chain.calls
puts = options_chain.puts
calls['Maturity'] = expiration_date

def to_ql_dates(date):
    date = pd.to_datetime(date)
    return ql.Date(date.day, date.month, date.year)

trade_dates = calls['lastTradeDate']
start_date, end_date = trade_dates.min(), trade_dates.max()
end_date = end_date + timedelta(days = 1)
prices = yf.download(ticker_symbol, start = start_date, end = end_date, progress = False)['Adj Close'].reset_index()
calls['lastTradeDate'] = calls['lastTradeDate'].apply(lambda x: x.strftime('%Y-%m-%d'))
prices['Date'] = prices['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
calls = calls.merge(prices, left_on = 'lastTradeDate', right_on = 'Date', how = 'left').drop('Date', axis = 1).rename( columns = {'Adj Close':'Price'})
calls['lastTradeDate'] = calls['lastTradeDate'].apply(to_ql_dates)
calls['Maturity'] = calls['Maturity'].apply(to_ql_dates)

spots = calls['Price'].values
strikes = calls['strike'].values
mkts = calls['lastPrice'].values
dividend = .0054
vols = calls['impliedVolatility'].values
calc = calls['lastTradeDate'].values
maturities = calls['Maturity'].values


results = [HestonParameters(spot_price, strike_price, market_price, dividend, traditional_implied_volatility, calculation_date, maturity_date) for 
spot_price, strike_price, market_price, traditional_implied_volatility, calculation_date, maturity_date in zip(spots, strikes, mkts, vols, calc, maturities)]
results_report = pd.concat(results, axis = 0)
error = results_report.groupby('Optimizer')['MSE'].sum()
reliability = results_report[results_report['Success'] == True].groupby('Optimizer').count() / (len(results_report)/2) * 100

64

In [82]:
import QuantLib as ql
import pandas as pd
from scipy.optimize import minimize

def HestonParameters(spot_price, strike_price, market_price, dividend_yield, traditional_implied_volatility, calculation_date, maturity_date, risk_free_rate = .00525, call_option = True):
    day_count = ql.Actual365Fixed()
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)

    # Set up the QuantLib environment
    ql.Settings.instance().evaluationDate = calculation_date
    if call_option:
        option_type = ql.Option.Call  # or ql.Option.Put for a put option
    else:
        option_type = ql.Option.Put
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    exercise = ql.EuropeanExercise(maturity_date)
    european_option = ql.VanillaOption(payoff, exercise)

    # Initial parameters for the Heston model
    initial_v0 = 0.1
    initial_kappa = 0.1
    initial_theta = traditional_implied_volatility
    initial_sigma = 0.1
    initial_rho = 0.1

    # Define the optimization objective function
    def objective_function(params):
        v0, kappa, theta, sigma, rho = params

        heston_process = ql.HestonProcess(
            ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count)),
            ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_yield, day_count)),
            ql.QuoteHandle(ql.SimpleQuote(spot_price)),
            v0, kappa, theta, sigma, rho
        )

        model = ql.HestonModel(heston_process)
        engine = ql.AnalyticHestonEngine(model)
        european_option.setPricingEngine(engine)

        model_price = european_option.NPV()
        error = (model_price - market_price) ** 2  # Squared error
        return error, model_price  # Return both error and the model price

    # Bounds for the parameters, excluding theta which is input by the user
    bounds = [(0.0001, 1.0), (0.0001, 2.0), (traditional_implied_volatility * 0.5, traditional_implied_volatility * 1.5), (0.0001, 1.0), (-0.999, 0.999)]

    # Initial parameter guesses, excluding theta which is input by the user
    initial_guess = [0.1, 0.1, traditional_implied_volatility, 0.1, 0.1]

    # Dictionary of optimizers to test, removing poor performers
    optimizers = {
        'L-BFGS-B': {'method': 'L-BFGS-B'},
        'TNC': {'method': 'TNC'},
    }

    # DataFrame to collect results
    results_df = pd.DataFrame(columns=['Optimizer', 'Success', 'Params', 'Objective_Value', 'Estimated_Price', 'Market_Price', 'MSE'])

    # Run optimizers and collect results
    for name, opt in optimizers.items():
        print(f"Running optimizer: {name}")
        
        # Run the optimizer and capture the additional returned value
        result = minimize(lambda x: objective_function(x)[0], initial_guess, method=opt['method'], bounds=bounds)
        
        success = result.success
        params = result.x if success else None
        # Now retrieve the full result from the objective function which includes the error and the model price
        objective_value, estimated_price = objective_function(params) if success else (None, None)
        error = (estimated_price - market_price) if success else None
        
        # Create a temporary DataFrame and concatenate it to the main DataFrame
        temp_df = pd.DataFrame({
            'Optimizer': [name],
            'Success': [success],
            'Params': [params],
            'Objective_Value': [objective_value],
            'Estimated_Price': [estimated_price],
            'Market_Price': [market_price if success else None],
            'MSE': [error**2]
        })
        results_df = pd.concat([results_df, temp_df], ignore_index=True)

    # Print the DataFrame
    return results_df.sort_values(by='MSE', ascending= True)


In [25]:
from datetime import datetime, timedelta
from scipy.optimize import basinhopping
import QuantLib as ql
import numpy as np

def optimize_params(market_price, strike_price, maturity, risk_free_rate, option_type, spot_price, current_date, yearly_historical_volatility):
    # Define bounds for the parameters
    bounds = [(0.0001, 5), (0.0001, 5), (0.0001, 5), (0.0001, 5), (-0.999, 0.999)]

    # Initial guess for the parameters based on historical volatility
    initial_params = [yearly_historical_volatility ** 2, 2.0, yearly_historical_volatility ** 2, 0.1, -0.7]

    # Objective function adapted for the single derivative
    def objective_function(params):
        return objective_function_single_derivative(params, market_price, strike_price, maturity, risk_free_rate, option_type, spot_price, current_date)

    # The basinhopping algorithm
    minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds}
    result = basinhopping(objective_function, initial_params, minimizer_kwargs=minimizer_kwargs)

    # Results
    optimized_params = result.x
    minimum_error = result.fun

    print('Minimum error:', minimum_error)
    print('Optimized parameters:', optimized_params)
    
    return optimized_params

def objective_function_single_derivative(params, market_price, strike_price, maturity, risk_free_rate, option_type, spot_price, current_date):
    v0, kappa, theta, sigma, rho = params

    # Check for parameter validity
    if v0 <= 0 or theta <= 0 or kappa <= 0 or sigma <= 0 or not (-1 < rho < 1) or 2 * kappa * theta <= sigma**2:
        return np.inf  # Return a "penalty" value indicating bad parameters

    model_price = heston_price_function(strike_price, spot_price, np.sqrt(v0), risk_free_rate, kappa, sigma, rho, theta, current_date, maturity, option_type)
    error = (model_price - market_price) ** 2
    return error

def heston_price_function(strike_price, spot_price, v0, risk_free_rate, kappa, sigma, rho, theta, current_date, maturity, option_type):
    # Convert the volatility input into variance
    variance = v0 ** 2
    
    # Calendar and Day count convention
    calendar = ql.UnitedStates()
    day_count = ql.Actual365Fixed()

    # Option type
    ql_option_type = ql.Option.Call if option_type.lower() == 'call' else ql.Option.Put
    
    # Create QuantLib dates from the current date and maturity
    valuation_date = ql.Date(current_date.day, current_date.month, current_date.year)
    maturity_date = valuation_date + int(maturity * 365.25)
    ql.Settings.instance().evaluationDate = valuation_date
    
    # Payoff and Exercise
    payoff = ql.PlainVanillaPayoff(ql_option_type, strike_price)
    exercise = ql.EuropeanExercise(maturity_date)

    # Heston process setup
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
    flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, risk_free_rate, day_count))
    dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, 0.0, day_count))
    heston_process = ql.HestonProcess(flat_ts, dividend_yield, spot_handle, variance, kappa, theta, sigma, rho)

    # Heston model setup
    heston_model = ql.HestonModel(heston_process)
    
    # Pricing engine
    engine = ql.AnalyticHestonEngine(heston_model)
    european_option = ql.VanillaOption(payoff, exercise)
    european_option.setPricingEngine(engine)

    # Price calculation
    try:
        price = european_option.NPV()
        return price
    except RuntimeError as e:
        print("Runtime error in pricing:", e)
        return np.inf  # Return a "penalty" value

# Example usage:
market_price = 10  # Example market price
strike_price = 100
maturity = 1  # Maturity in years
risk_free_rate = 0.01  # Risk-free rate
option_type = 'call'  # 'call' or 'put'
spot_price = 100
current_date = datetime.today()  # Replace with your actual date object
yearly_historical_volatility = 0.20

# Optimize parameters
optimized_parameters = optimize_params(
    market_price, strike_price, maturity, risk_free_rate,
    option_type, spot_price, current_date, yearly_historical_volatility
)
